In [ ]:
import glob
import rasterio
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from rasterio.plot import show
import gdal, ogr, os, osr
from skimage import exposure
import geopandas as gpd
from rasterio.mask import mask

# Importing modules for merging rasters to form a true colour image
from rasterio.merge import merge
from skimage import exposure

# Importing Machine Learning modules for splitting splittng, training testing and classifying the data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
ImgList = glob.glob('/home/emmanuel/Desktop/ML/LC081720712019031701T1-SC20191027162326/*band*tif')
shapes = glob.glob('/home/emmanuel/Desktop/ML/Landsat8Shapes/*shp')
ImgList
#shapes

In [ ]:
blue_path = ImgList[6]
green_path = ImgList[4]
red_path = ImgList[2]
nir_path = ImgList[0]

In [ ]:
blue_raster = rasterio.open(blue_path)
green_raster = rasterio.open(green_path)
red_raster = rasterio.open(red_path)
nir_raster = rasterio.open(nir_path)

In [ ]:
blue_array = blue_raster.read(1)*0.0001
green_array = green_raster.read(1)*0.0001
red_array = red_raster.read(1)*0.0001
nir_array = nir_raster.read(1)*0.0001
nir_array.max()

In [ ]:
def Landsat8_Cleaner(array):
    better_array = np.where((array <= 1.0) & (array >= 0), array, 0.0)
    return better_array

In [ ]:
cleaned_blue = Landsat8_Cleaner(blue_array)

In [ ]:
cleaned_blue[2000:3000, 2000:3000].max()

In [ ]:
plt.imshow(cleaned_blue)#.max()

In [ ]:
cleaned_blue = Landsat8_Cleaner(blue_array)
cleaned_green = Landsat8_Cleaner(green_array)
cleaned_red = Landsat8_Cleaner(red_array)
cleaned_nir = Landsat8_Cleaner(nir_array)
print(green_array.max(), green_array.min(), np.nanmax(cleaned_green), np.nanmin(cleaned_green))
plt.imshow(cleaned_blue)
cleaned_red.max()

In [ ]:
# Function that turns an array into a raster
def array2raster(rasterfn,newRasterfn,array):
    raster = gdal.Open(rasterfn)
    geotransform = raster.GetGeoTransform()
    originX = geotransform[0]
    originY = geotransform[3]
    pixelWidth = geotransform[1]
    pixelHeight = geotransform[5]
    cols = raster.RasterXSize
    rows = raster.RasterYSize
    driver = gdal.GetDriverByName("GTiff")
    outRaster = driver.Create(newRasterfn, cols, rows, 1, gdal.GDT_Float32)
    outRaster.SetGeoTransform((originX, pixelWidth, 0, originY, 0, pixelHeight))
    outband = outRaster.GetRasterBand(1)
    outband.WriteArray(array)
    outRasterSRS = osr.SpatialReference()
    outRasterSRS.ImportFromWkt(raster.GetProjectionRef())
    outRaster.SetProjection(outRasterSRS.ExportToWkt())
    outband.FlushCache()

In [ ]:
def IndexCalc(array1, array2):
    np.seterr(divide = 'ignore', invalid = 'ignore')
    Index = (array1 - array2)/(array1 + array2)
    return Index

In [ ]:
IndexCalc(cleaned_nir, cleaned_red)

In [ ]:
def IndexCleaner(index):
    #better_index = np.where((index <= 1) & (index >= -1), index, np.nan)
    better_index1 = pd.DataFrame(index)
    better_index2 = better_index1.replace(-0.0, 0.0)
    better_index3 = np.array(better_index2)
    return better_index3


index1 = np.array([1, -1, 2, 0.9, -0.1, -8, -1, 0.4, 0.4, -0.0, 89])
IndexCleaner(index1)

In [ ]:
rasterpath = '/home/emmanuel/Desktop/ML/LC081720712019031701T1-SC20191027162326/LC08_L1TP_172071_20190317_20190325_01_T1_sr_band5.tif'

In [ ]:
def CreatRasters(blue_array, green_array, red_array, nir_array, rasterpath):
    array2raster(rasterpath, rasterpath[:-9] + 'B02.tif', blue_array)
    array2raster(rasterpath, rasterpath[:-9] + 'B03.tif', green_array)
    array2raster(rasterpath, rasterpath[:-9] + 'B04.tif', red_array)
    array2raster(rasterpath, rasterpath[:-9] + 'B05.tif', nir_array)
    
    NDVI = IndexCalc(nir_array, red_array)
    NDVI = IndexCleaner(NDVI)
    array2raster(rasterpath, rasterpath[:-9] + 'NDVI.tif', NDVI)
    
    NDWI = IndexCalc(green_array, nir_array)
    NDWI = IndexCleaner(NDWI)
    array2raster(rasterpath, rasterpath[:-9] + 'NDWI.tif', NDWI)

In [ ]:
CreatRasters(cleaned_blue, cleaned_green, cleaned_red, cleaned_nir, rasterpath)

In [ ]:
blue_raster = rasterio.open(rasterpath[:-9] + 'B02.tif')
green_raster = rasterio.open(rasterpath[:-9] + 'B03.tif')
red_raster = rasterio.open(rasterpath[:-9] + 'B04.tif')
nir_raster = rasterio.open(rasterpath[:-9] + 'B05.tif')
NDVI_raster = rasterio.open(rasterpath[:-9] + 'NDVI.tif')
NDVI_array = IndexCleaner(NDVI_raster.read(1))
NDWI_raster = rasterio.open(rasterpath[:-9] + 'NDWI.tif')
NDWI_array = IndexCleaner(NDWI_raster.read(1))

In [ ]:
plt.figure(figsize = (15,15))
show(nir_raster)

In [ ]:
NDVI_array.min()

In [ ]:
np.nanmin(NDVI_array)

In [ ]:
# Get shape geometry in json format
def getFeatures(gdf):
    '''Function to parse features from GeoDataFrame in such a manner that rasterio can take them'''
    import json
    return [json.loads(gdf.to_json())['features'][0]['geometry']]

In [ ]:
def getClasses(shp, blue, green, red, nir, NDVI, NDWI):
    class_df = pd.DataFrame()
    df_ls = []
    geom = gpd.read_file(shp)
    geom = geom.to_crs(crs = red.crs.data)
    sa = np.arange(len(geom))
    sb = np.arange(1, len(geom)+1)
    for a, b in zip(sa, sb):
        cord1 = getFeatures(geom.iloc[a:b,:])
        out_blue, out_transform = mask(blue, cord1, crop = True)
        out_green, out_transform1 = mask(green, cord1, crop = True)
        out_red, out_transform2 = mask(red, cord1, crop = True)
        out_nir, out_transform3 = mask(nir, cord1, crop = True)
        out_ndvi, out_transform4 = mask(NDVI, cord1, crop = True)
        out_ndwi, out_transform5 = mask(NDWI, cord1, crop = True)
        
        df_ls.append(pd.DataFrame({'Blue_Band': out_blue.ravel(),
                                  'Green_Band': out_green.ravel(),
                                  'Red_Band': out_red.ravel(),
                                  'NIR_Band': out_nir.ravel(),
                                  'NDVI_Band': out_ndvi.ravel(),
                                  'NDWI_Band': out_ndwi.ravel(),
                                  'Class': geom.class1[0]}))
    class_df = pd.concat(df_ls, axis = 0)
    class_df = class_df.replace(0.0, np.nan)
    return class_df.dropna()

In [ ]:
red_raster.read(1)[2000:3000, 2000:3000]

In [ ]:
#===========================================================================================

# Gettomg all the image pixels
def allPixels(blue, green, red, nir, NDVI, NDWI):
    all_df = pd.DataFrame()
    df_ls = []
    
    df_ls.append(pd.DataFrame({'Blue_Band':blue.read(1)[5000:7000, 4000:6000].ravel(),
                              'Green_Band':green.read(1)[5000:7000, 4000:6000].ravel(),
                              'Red_Band':red.read(1)[5000:7000, 4000:6000].ravel(),
                              'NIR_Band':nir.read(1)[5000:7000, 4000:6000].ravel(),
                              'NDVI_Band':NDVI.read(1)[5000:7000, 4000:6000].ravel(),
                              'NDWI_Band':NDWI.read(1)[5000:7000, 4000:6000].ravel()}))
    all_df = pd.concat(df_ls, axis = 0)
    return all_df

#===========================================================================================

In [ ]:
# For loop for getting pixel values and classes
cls_df = pd.DataFrame()
df_ls = []
for s in shapes:
    df_ls.append(getClasses(s, blue_raster, green_raster, red_raster, nir_raster, NDVI_raster, NDWI_raster))
cls_df = pd.concat(df_ls, axis = 0)
#===========================================================================================

In [ ]:
#===========================================================================================
# SPliting our data into training and testing sets
X = cls_df.iloc[:,:6].values
y = cls_df.iloc[:,6].values
X_train1, X_test1, y_train1, y_test1 = train_test_split(X, y, test_size = 0.3, random_state = 5)

clf = RandomForestClassifier(n_estimators = 500, max_depth = 20)
clf.fit(X_train1, y_train1)
y_pred = clf.predict(X_test1)
#============================================================================================

In [ ]:
import pickle
filename = 'Landsat8_Classifier.sav'
pickle.dump(clf, open(filename, 'wb'))

In [ ]:
print(accuracy_score(y_test1, y_pred))

In [ ]:
cdf = allPixels(blue_raster, green_raster, red_raster, nir_raster, NDVI_raster, NDWI_raster)
img = cdf.iloc[:,:].values
cdf

In [ ]:
img[1:].min()

In [ ]:
pred_class = clf.predict(img)

In [ ]:
landclass = pred_class.reshape(2000,2000)

In [ ]:
plt.figure(figsize=(15,15))
plt.imshow(cleaned_red[5000:7000, 4000:6000])

In [ ]:
landclass = np.array(landclass, dtype = float)
classes = ['Clouds','Shadows','nodata']
length = len(classes)
plt.figure(figsize=(15,15))
cmap = plt.cm.get_cmap("Set1", length)
plt.imshow(landclass, cmap=cmap)
ticks = np.arange(1, length + 1)
colorbar = plt.colorbar(ticks = ticks, orientation="horizontal")
#colorbar = plt.colorbar(ticks=ticks, orientation = "horizontal")
colorbar.set_ticklabels(classes)
plt.savefig('Classified1.png')

In [ ]:
plt.imshow(landclass == 6)

In [ ]:

NDVI_array = IndexCleaner(NDVI_array)
colormap = plt.cm.nipy_spectral
fig = plt.figure(figsize = (20,20))
plt.rc('ytick', labelsize= 20)
plt.rc('xtick', labelsize = 20)
plt.title('Enhanced Vegetation Index of\n Landsat-8 Image\n', fontsize = 20, fontweight = 'bold')
plt.imshow(NDVI_array, cmap = colormap)
plt.colorbar(orientation = 'vertical', shrink = 0.67)
plt.savefig('NDVI.png')

In [ ]:
import numpy as np
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sns.set(context = 'notebook', style = 'white', rc = {'figure.figsize':(14,10)})

In [ ]:
digits = load_digits()

In [ ]:
import umap

In [ ]:
trans = umap.UMAP(n_neighbors = 5, random_state = 42)